In [ ]:
import numpy as np
import math

from collections import deque

In [ ]:
# Define a Class Atom with x,y,z and positive boolean
class Atom:
    def __init__(self, x, y, z, positive):
        self.x = x
        self.y = y
        self.z = z
        self.positive = positive

# Define a Goal Class as list of Atoms
class Goal:
    def __init__(self, atoms):
        self.atoms = atoms

    # Append
    def append(self, atom):
        self.atoms.append(atom)

    # print
    def print(self):
        # for each atom in the goal
        for atom in self.atoms:
            # print the atom
            print(atom.x, atom.y, atom.z, end="")
            if atom.positive:
                print(" +")
            # if the atom is negative, print a -
            else:
                print(" -")
            

class GoalBag:
    def __init__(self, goals):
        self.goals = goals

    # Append
    def append(self, goal):
        self.goals.append(goal)

    
    # print
    def print(self):
        for goal in self.goals:
            print("--------------------")
            goal.print()
            print("--------------------")


The larger d is, the larger the search space and complexity of the problem, so d can be limited to allow an acceptable amount of approximation error such that the resulting program is shorter and computational expense of compiling it is reduced.

In [ ]:
d = 2

kernel = np.array([
  [ 0,  0,  0],
  [-3,  1,  0],
  [-3,  0,  2]
]) * (1/pow(2,d))

kernels = [
    np.array([
  [ 0,  0,  0],
  [-3,  1,  0],
  [-3,  0,  2]
]) * (1/pow(2,d))
,
np.array([
  [ -4,  -1,  -1],
  [-1,  2,  0],
  [1,  1,  0]
]) * (1/pow(2,d))
,
np.array([
  [ -1,  2,  0],
  [-1,  1,  -3],
  [0,  -3,  0]
]) * (1/pow(2,d))

]

print(kernels)


[array([[ 0.  ,  0.  ,  0.  ],
       [-0.75,  0.25,  0.  ],
       [-0.75,  0.  ,  0.5 ]]), array([[-1.  , -0.25, -0.25],
       [-0.25,  0.5 ,  0.  ],
       [ 0.25,  0.25,  0.  ]]), array([[-0.25,  0.5 ,  0.  ],
       [-0.25,  0.25, -0.75],
       [ 0.  , -0.75,  0.  ]])]


In [ ]:
def kernel_to_goal(kernel):
    nrows, ncols = kernel.shape
    scaled_kernel = kernel
    
    # Goal is a class
    goal = Goal([])

    # Iterate through the kernel elements and create Atoms accordingly
    for y in range(nrows):
        for x in range(ncols):
            value = scaled_kernel[y, x]

            # get the nearest integer multiple of 1/2^d of the value
            value = round(value * pow(2,d))

            # Get the sign
            sign = '+' if value > 0 else '-'
        
            dx = x - math.floor(ncols/2)
            dy = math.floor(nrows/2) - y

            for _ in range(abs(value)):
                # Use atom class
                atom = Atom(dx, dy, 0, sign == '+')

                # Add atom to goal
                goal.append(atom)


    return goal


In [ ]:
goal = kernel_to_goal(kernel)
goal.print()

# for each kernel in kernels create a goal and add it to the goal bag
F_goal_bag = GoalBag([])
for kernel in kernels:
    goal = kernel_to_goal(kernel)
    F_goal_bag.goals.append(goal)

# print the goal bag
F_goal_bag.print()

-1 0 0 -
-1 0 0 -
-1 0 0 -
0 0 0 +
-1 -1 0 -
-1 -1 0 -
-1 -1 0 -
1 -1 0 +
1 -1 0 +
--------------------
-1 0 0 -
-1 0 0 -
-1 0 0 -
0 0 0 +
-1 -1 0 -
-1 -1 0 -
-1 -1 0 -
1 -1 0 +
1 -1 0 +
--------------------
--------------------
-1 1 0 -
-1 1 0 -
-1 1 0 -
-1 1 0 -
0 1 0 -
1 1 0 -
-1 0 0 -
0 0 0 +
0 0 0 +
-1 -1 0 +
0 -1 0 +
--------------------
--------------------
-1 1 0 -
0 1 0 +
0 1 0 +
-1 0 0 -
0 0 0 +
1 0 0 -
1 0 0 -
1 0 0 -
0 -1 0 -
0 -1 0 -
0 -1 0 -
--------------------


In [ ]:
K_ID = np.array([
    [ 0,  0,  0],
    [ 0,  pow(2,d),  0],
    [ 0,  0,  0]
]) * (1/pow(2,d))

K_ID

array([[0., 0., 0.],
       [0., 1., 0.],
       [0., 0., 0.]])

In [ ]:
G_ID = kernel_to_goal(K_ID)

G_ID.print()

0 0 0 +
0 0 0 +
0 0 0 +
0 0 0 +


In [ ]:
def child_generation(n):
    inst = 'add'

    goal = n.goals[0].print()
    print(goal)

    return None

s = F_goal_bag
# Array of touple (parent, child)
deque = deque([(s, None)])

while deque:
    (n,g) = deque.popleft()
    
    if g == None:
        g = child_generation(n)

    c = g

    if c != None:
        deque.appendleft((c, None))
        deque.append((n, g))
    else:
        break



-1 0 0 -
-1 0 0 -
-1 0 0 -
0 0 0 +
-1 -1 0 -
-1 -1 0 -
-1 -1 0 -
1 -1 0 +
1 -1 0 +
None


In [ ]:
def multiplicity(a, G):
    # Assuming an implementation for counting the number of times 'a' appears in 'G'
    # Example: return G.count(a)
    pass


def total_cost(C):
    return sum([len(G) for G in C])

def dists_cost(C):
    cost = 0
    for G in C:
        for a in G:
            cost += (abs(a[0]) + abs(a[1]) + (-1 if a[3] == '-' else 0)) * s(G, C)
    return cost

def s(G, C):
    B = next((B for B in C if G.issubset(B)), None)
    if B is None:
        return 1
    else:
        return 0.5

def n(a):
    return 1 if a[3] == '-' else 0

def reps_cost(C):
    cost = 0
    for G in C:
        unique_translated = True
        if unique_translated:
            num_atoms = len(G)
            pair_exists = any(a != b for a in G for b in G)

            cost += num_atoms / 2 if pair_exists else 0

    return cost

def divs_cost(C, d=2):
    return 2 * d * min(multiplicity(a, G) for G in C for a in G)

def cost(C):
    return total_cost(C) + dists_cost(C) + reps_cost(C) + divs_cost(C)